In [5]:
from langchain.llms import CTransformers
from langchain.chains import LLMChain
from langchain import PromptTemplate

import os
import io
import gradio as gr
import time


In [14]:
import torch

In [ ]:
custom_prompt_template = """
You are an AI coding assistant and your task is to solve coding problems and return code snippets based on given user's query. Below is the user's query.
Query: {query}
You just return the helpful code and related details.
Helpful code and related details:
"""

def set_custom_prompt():
    prompt = PromptTemplate(
        template=custom_prompt_template,
        input_variables = ['query'],
    )
    return prompt

# llama_model = torch.load('D:\Llama\CodeLlama-7b-Python\consolidated.00.pth', map_location=torch.device('cpu'))

def load_model():
    llm = CTransformers(
        # model = llama_model,
        model = ['D:\Llama\CodeLlama-7b-Python'],
        model_type = "llama",
        max_new_tokens = 1096,
        temperature = 0.2,
        repetition_penalty = 1.13
    )
    return llm


def chain_pipeline():
    llm = load_model()
    qa_prompt = set_custom_prompt()
    qa_chain = LLMChain(prompt=qa_prompt, llm=llm)
    return qa_chain

llmchain = chain_pipeline()

def bot(query):
    llm_response = llmchain.process(query)
    return llm_response

with gr.Blocks(title="Code Llama Demo") as demo:
    gr.Markdown("## Code Llama Demo")
    chatbot = gr.Chatbot([], elem_id="chatbot", height=500, placeholder="Ask me anything about coding!")
    msg = gr.Textbox(lines=5, label="Response", readonly=True)
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = bot(message)
        chat_history.append(message, bot_message)
        time.sleep(2)
        return "", chat_history
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()

In [2]:
from transformers import AutoTokenizer
import transformers
import torch

from accelerate import Accelerator

acc = Accelerator()

model = "d:/Llama/CodeLlama-7b-Python"

torch.set_default_device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)
model_config = transformers.AutoConfig.from_pretrained(
    model,
    #use_auth_token=HF_AUTH
)

c:\Users\wangqian\flaskProject1.git\KNOWweb\PyT\lib\site-packages\transformers\models\auto\tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [3]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    # config = model_config,
    torch_dtype=torch.float16,
    device_map="auto"
    # offload_folder = "offload",
    # offload_state_dict=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
def get_response(prompt: str) -> None:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256
    )
    print("Chatbot:", sequences[0]['generated_text'].todevice(torch.device('cpu')))


In [5]:
get_response("How do I")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'